# FFT(Fast Fourier Transform)를 이용한 고속 다항식 곱셈 보고서

## 목차
- I. 서론: 다항식 곱셈의 계산 복잡도 분석  
- II. 다항식 곱셈의 구조  
  - II-1. 다항식의 두 가지 표현법  
  - II-2. 짝수홀수 분할의 구조적 최적화와 재귀 구조  
- III. DFT, FFT, IFFT의 정의 및 성질  
  - III-1. DFT, FFT, IFFT의 개념  
  - III-2. DFT가 성립하기 위한 조건  
  - III-3. FFT와 IFFT가 동일한 이유  
- IV. 단위근  
  - IV-1. primitive 단위근  
  - IV-2. 단위근의 대칭성  
  - IV-3. even/odd($x^2$)에서 보존되는 단위근의 재귀적 성질  
- V. 구현
  - V-1. Cooley–Tukey 알고리즘
  - V-2. 알고리즘 의사 코드 (Pseudo-code)
  - V-3. 재귀 호출 흐름 시각화
  - V-4. 파이썬 구현: FFT와 IFFT

---

## I. 서론: 다항식 곱셈의 계산 복잡도 분석

두 다항식 $A(x), B(x)$의 곱 $C(x) = A(x)B(x)$를 직접 계산하면 $\mathcal{O}(n^2)$의 복잡도를 갖는다.  
그러나 FFT(고속 푸리에 변환)를 이용하면 $\mathcal{O}(n \log n)$의 시간에 계산이 가능하다.  
이는 다음의 변환 과정으로 가능하다:
#### FFT,IFFT를 적용한 다항식 곱셈 흐름도:
![fft](../image/fft.png)
1. $A(x), B(x)$를 점값 표현으로 변환 → FFT  
2. 점값에서 대응 항끼리 곱함  
$$
C(x_i) = A(x_i) \times B(x_i)
$$
3. 다시 계수 표현으로 복원 → IFFT  

FFT와 IFFT는 다항식의 계수 표현과 점값 표현 간 변환을 효율적으로 수행하는 알고리즘이다.  
- **DFT(Discrete Fourier Transform)**: 계수 벡터를 주어진 단위근을 기반으로 점값 벡터로 변환하는 선형 변환으로,  
  주어진 다항식 $P(x) = \sum_{j=0}^{n-1} a_j x^j$에 대해 $n$차 단위근 $\omega_n = e^{-2\pi i / n}$을 기준으로 다음과 같이 정의된다:  
  $$
  \text{DFT}_n[P](\omega_n^k) = \sum_{j=0}^{n-1} a_j \cdot \omega_n^{jk}
  $$ 
- **FFT(Fast Fourier Transform)**: DFT를 빠르게 계산하는 알고리즘이다.
- **IFFT(Inverse FFT)**: 점값 표현을 다시 계수 표현으로 복원하는 변환이다.

이들 변환의 상세한 정의와 성질은 III장에서 다룬다.

---

## II. 다항식 곱셈의 구조

### II-1) 다항식의 두 가지 표현법

- **계수 표현 (Coefficient form)**  
  $A(x) = a_0 + a_1x + a_2x^2 + \cdots + a_{n-1}x^{n-1}$

- **점값 표현 (Point-value form)**  
  $\{ (x_0, A(x_0)), (x_1, A(x_1)), \dots, (x_{n-1}, A(x_{n-1})) \}$

> $n$개의 서로 다른 점에서의 값이 주어지면 $n-1$차 다항식을 유일하게 복원할 수 있다.

### 점값 표현의 유일성 증명

> **$d+1$개의 서로 다른 점을 모두 지나는 $d$차 다항식은 오직 하나만 존재한다.**

![d+1](../image/d+1.png)
위와 같이 4개의 점이 주어졌을때 이 점을 모두 통과하는 3차 이하의 다항식은 하나만 존재한다. 이 성질은 차수가 3일 때 뿐 아니라 항상 적용되며 증명은 다음과 같다:

$d$차 다항식 $P(x)$와 $Q(x)$가 서로 다른 $d+1$개의 점 $(x_0, y_0), (x_1, y_1), \dots, (x_d, y_d)$을 모두 지난다고 가정하자.

즉, 모든 $i=0$부터 $d$까지에 대해 다음이 성립한다:

$$
P(x_i) = y_i,\quad Q(x_i) = y_i
$$

두 다항식의 차를 $R(x) = P(x) - Q(x)$라고 정의하자.

그러면 $R(x)$는 $d$차 이하의 다항식이면서 다음을 만족한다:

$$
R(x_i) = 0 \quad \text{for all } i = 0, 1, \dots, d
$$

즉, $R(x)$는 서로 다른 $d+1$개의 점에서 0이 된다.

그러나 $d$차 이하의 다항식은 **최대 $d$개의 서로 다른 근**만 가질 수 있으므로, $R(x)$가 $d+1$개의 서로 다른 점에서 0이 되는 것은 **불가능**하다. (단, $R(x) \equiv 0$인 경우 제외)

따라서 $R(x) \equiv 0$, 즉 $P(x) = Q(x)$여야 한다.

**결론:** 서로 다른 $d+1$개의 점을 모두 지나는 $d$차 다항식은 유일하다.

이 성질은 FFT를 기반으로 한 다항식 곱셈에서 핵심이 된다.

FFT에서는 $n$차 미만 다항식 두 개를 곱하면 $2n-1$차 미만의 결과 다항식이 생성되므로, $2n$개의 서로 다른 점에서 값을 알면 결과 다항식을 완전히 복원할 수 있다.  
따라서 $n$차 미만 다항식을 점값 표현으로 바꾸기 위해 $2n$개의 입력 점(일반적으로 $n$개의 $n$차 미만 계수에 대해 $2n$개의 $n$차 단위근)을 사용하며, 이 점들에서의 값을 FFT로 계산한다.  
곱셈은 점값끼리의 곱으로 이루어지고, 다시 $2n$개의 점값을 기준으로 IFFT를 수행하면 계수 표현으로 되돌릴 수 있다.

---

### II-2) 짝수/홀수 분할의 구조적 최적화와 재귀 구조

#### 1. 재귀 분해 구조: 짝수/홀수 항 나누기

![even vs odd](../image/even_odd.png)

다항식 $P(x)$가 계수 $a_i$를 가질 때, 짝수 차수 항과 홀수 차수 항을 분리하면 함수의 대칭성을 활용한 구조로 나타낼 수 있다.  
이때, 다항식은 다음과 같이 정의된다:

$P(x) = \sum_{i=0}^{n-1} a_i x^i = a_0 + a_1 x + a_2 x^2 + \cdots + a_{n-1} x^{n-1}$

짝수 차수 항들로 구성된 다항식:

$P_{\text{even}}(x) = \sum_{j=0}^{n/2 - 1} a_{2j} x^j$

홀수 차수 항들로 구성된 다항식:

$P_{\text{odd}}(x) = \sum_{j=0}^{n/2 - 1} a_{2j+1} x^j$

따라서 원래의 다항식 $P(x)$는 다음과 같이 분해된다:

$P(x) = P_{\text{even}}(x^2) + x \cdot P_{\text{odd}}(x^2)$

이 구조는 FFT의 재귀 분할 구조를 형성한다.  
특히, $P(x)$를 $P_{\text{even}}(x^2)$와 $x \cdot P_{\text{odd}}(x^2)$로 분리함으로써, DFT 계산을 절반 크기의 부분 문제로 나눌 수 있다.

---

#### 2. 버터플라이 구조: 계산량 절반으로 줄이기

다항식 $P(x)$를 DFT로 계산한다고 할 때, 평가할 점이 $\omega^k$라면:

$\text{DFT}_n[P](\omega^k) = P(\omega^k) = P_{\text{even}}(\omega^{2k}) + \omega^k \cdot P_{\text{odd}}(\omega^{2k})$

여기서 $\omega$는 $n$차 단위복소근이고, $(\omega^k)^2 = \omega^{2k}$이므로,  
$\omega^2$는 $n$이 짝수일 때 $n/2$차 단위복소근이 된다.  
따라서 위 식은 $\omega^2$를 새로운 단위근으로 하는 길이가 $n/2$인 DFT($n/2$-점 DFT)로 해석할 수 있다:

- $P_{\text{even}}$을 $\omega^{2k}$에 대해 평가 → $n/2$-점 DFT  
- $P_{\text{odd}}$도 $\omega^{2k}$에 대해 평가 → $n/2$-점 DFT

---

![butterfly structure](../image/butterfly_structure.png)

---

이처럼 $\omega^k$에 대해 구하고자 했던 $\text{DFT}_n[P](\omega^k)$는  
짝수/홀수로 나눠진 두 다항식의 절반 크기 DFT 결과를 바탕으로 다음과 같은 두 값을 한 번에 계산할 수 있다:

$\text{DFT}_n[P](\omega^k) = E_k + \omega^k \cdot O_k$  
$\text{DFT}_n[P](\omega^{k + n/2}) = E_k - \omega^k \cdot O_k$

여기서 $E_k = P_{\text{even}}(\omega^{2k})$, $O_k = P_{\text{odd}}(\omega^{2k})$는  
각각 $n/2$-점 DFT 결과로 얻어진 값이다.  
즉, 한 번의 계산으로 짝을 이루는 두 점에서의 DFT 결과를 동시에 얻을 수 있다.

이 구조를 butterfly 구조라고 하며, $+\omega^k \cdot O_k$와 $-\omega^k \cdot O_k$ 조합으로 구성된다.  
화살표가 나타내듯, $E_k$ 값은 두 결과에 공통으로 더해지고,  
$O_k$ 값은 $\pm \omega^k$로 곱해져 각각 더해지거나 빼진다.

이로 인해 각 단계마다 계산량은 총 $n$이 필요하고 깊이는 $\log n$이 되므로 전체 FFT 계산량은 $O(n \log n)$으로 감소한다.


#### 재귀 흐름도:
![recursive](../image/recurse.png)    
- 입력 다항식 $P(x)$  
- 짝수 차항: $P_{\text{even}}(x^2)$, 홀수 차항: $P_{\text{odd}}(x^2)$  
- 재귀적으로 FFT 수행  
- 점값 계산: $P_{\text{even}}(w^2)$, $P_{\text{odd}}(w^2)$  
- 병합: $P(w) = P_{\text{even}}(w^2) + w \cdot P_{\text{odd}}(w^2)$

---

#### 단위근의 필요성

##### $n$ 깊이 재귀에서 pair의 보존 필요 → 실수(real number)로는 불가능

- 제곱했을 때 원점 대칭이 보존되어야함
- $n = 8$일 때, 재귀를 통해 $n/2$, $n/4$ 등으로 나누어 계산
- 각 단계에서 분할된 even/odd를 합칠 때 복소수 회전의 특성이 필요
- 실수만으로는 $w_n^k$의 회전 성질을 보존할 수 없음

---

## III. DFT, FFT, IFFT의 정의 및 성질

### III-1.DFT, FFT, IFFT의 개념

#### DFT(Discrete Fourier Transform) (이산 푸리에 변환)
- 주어진 길이 $n$의 복소수열 $\{a_0, a_1, \dots, a_{n-1}\}$를 복소수 주파수 성분으로 분해하는 선형 변환
- 다항식을 점값 표현으로 변환하여 $A(w_n^k)$ 꼴로 표현 가능하게 함

#### FFT(Fast Fourier Transform) (고속 푸리에 변환)
- DFT를 $\mathcal{O}(n^2)$ 대신 $\mathcal{O}(n \log n)$ 시간에 수행하는 재귀적 알고리즘

#### IFFT(Inverse Fast Fourier Transform) (역 이산 푸리에 변환)
- DFT의 역변환, 점값 표현을 계수 표현으로 복원


### III-2. DFT가 성립하기 위한 조건

길이가 $n$인 복소수 벡터 $\mathbf{a} = (a_0, a_1, \dots, a_{n-1})$에 대해, DFT는 다음과 같은 수식을 따른다.

$$
A_k = \sum_{j=0}^{n-1} a_j \cdot \omega_n^{jk} \quad (0 \leq k < n)
$$

여기서 $\omega_n = e^{-2\pi i / n}$은 $n$번째 **단위근**이다.  
이때 $\omega_n^n = 1$을 만족하고, $\omega_n^k \neq 1$ for $0 < k < n$ 인 경우를 **primitive**한 단위근이라고 한다.

> **DFT가 성립하기 위한 조건**
> - 변환에 사용되는 $\omega_n$은 $n$번째 단위근이어야 한다.
> - 인덱스 $jk$는 모듈로 $n$에 따라 해석되므로 **인덱스 정렬과 순서**에 유의해야 한다.

> **단위근의 의미**  
> 복소평면 상에서 $\omega_n = e^{-2\pi i / n}$는 단위원의 원을 $n$등분하는 각도를 의미한다.  
> 즉, $\omega_n^k$는 반지름 1인 원에서 $k$번째 점을 가리키며, 이는 주기적 회전의 성질을 내포한다.

---
### III-3. FFT와 IFFT가 동일한 이유  

FFT는 다음과 같이 다항식을 점값으로 변환하는 DFT 행렬 곱 형태로 표현할 수 있다:

$P(x) = p_0 + p_1x + p_2x^2 + \cdots + p_{n-1}x^{n-1}$ 에 대해,

$$
\begin{bmatrix}
P(w^0) \\
P(w^1) \\
P(w^2) \\
\vdots \\
P(w^{n-1})
\end{bmatrix}
=
\begin{bmatrix}
1 & 1 & 1 & \cdots & 1 \\
1 & w & w^2 & \cdots & w^{n-1} \\
1 & w^2 & w^4 & \cdots & w^{2(n-1)} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & w^{n-1} & w^{2(n-1)} & \cdots & w^{(n-1)^2}
\end{bmatrix}
\begin{bmatrix}
p_0 \\
p_1 \\
p_2 \\
\vdots \\
p_{n-1}
\end{bmatrix}
$$

여기서 $w = e^{\frac{2\pi i}{n}}$ 는 $n$번째 단위근이다. 위 행렬은 DFT 행렬 $\mathbf{F}_n$이라 부르며, 이를 통해 점값 표현으로의 변환이 이루어진다.

---

이제 IFFT는 이 DFT 행렬의 역행렬을 통해 계수로 되돌리는 연산이다:

$$
\begin{bmatrix}
p_0 \\
p_1 \\
p_2 \\
\vdots \\
p_{n-1}
\end{bmatrix}
=
\mathbf{F}_n^{-1}
\begin{bmatrix}
P(w^0) \\
P(w^1) \\
P(w^2) \\
\vdots \\
P(w^{n-1})
\end{bmatrix}
$$

그리고 $\mathbf{F}_n^{-1}$은 다음과 같이 나타낼 수 있다:

$$
\mathbf{F}_n^{-1} = \frac{1}{n}
\begin{bmatrix}
1 & 1 & 1 & \cdots & 1 \\
1 & w^{-1} & w^{-2} & \cdots & w^{-(n-1)} \\
1 & w^{-2} & w^{-4} & \cdots & w^{-2(n-1)} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & w^{-(n-1)} & w^{-2(n-1)} & \cdots & w^{-(n-1)^2}
\end{bmatrix}
$$

즉, IFFT 역시 DFT 구조를 유지한 채, 단위근을 켤레 복소수로 바꾸고 전체에 $\frac{1}{n}$을 곱하는 방식으로 계산된다. 이는 DFT의 **conjugate transpose가 역행렬**이 됨을 의미한다.

---

## IV. 단위근
### 단위근의 특성:
![w^k](../image/w^k.png)
### IV-1) primitive 단위근
- $w_n$이 primitive root가 되려면:
  - $w_n^n = 1$
  - $w_n^k \neq 1$ for $0 < k < n$

→ $n$개의 고유한 회전 각도를 제공하며, $n$차 다항식을 고르게 분해 가능

### IV-2) 단위근의 대칭성
- Figure 2: 단위근 $w_n^k = e^{-\frac{2\pi i k}{n}}$는 복소평면의 단위원 상에서 $2\pi k/n$ 만큼 균등하게 회전하며 배치된다.
- 앞의 절반(즉, $k = 0, 1, \dots, \frac{n}{2}-1$) 단위근에 마이너스를 붙인 값들이 뒤의 절반($k = \frac{n}{2}, \frac{n}{2}+1, \dots, n-1$) 단위근과 정확히 일치한다.

$$
\begin{aligned}
-w_n^k &= -e^{-\frac{2\pi i k}{n}} \\
&= e^{i \pi} \cdot e^{-\frac{2\pi i k}{n}} \quad \text{(since } -1 = e^{i \pi} \text{)} \\
&= e^{-\frac{2\pi i k}{n} + i \pi} \\
&= e^{-\frac{2\pi i k}{n} + \frac{2\pi i n}{2n}} \quad \text{(rewrite } i \pi = \frac{2\pi i n}{2n} \text{)} \\
&= e^{-\frac{2\pi i k}{n} + \frac{2\pi i (n/2)}{n}} \\
&= e^{-\frac{2\pi i}{n}(k - \frac{n}{2})} \\
&= w_n^{k + \frac{n}{2}} \quad \text{(since } w_n^{m} = e^{-\frac{2\pi i m}{n}} \text{)}
\end{aligned}
$$

- 따라서,

$$
-w_n^k = w_n^{k + \frac{n}{2}} \quad \text{for } 0 \leq k < \frac{n}{2}
$$

- 이 결과는 FFT에서 짝수 인덱스와 홀수 인덱스의 분할 후 재조합할 때, 절반 길이 단위근들만으로 전체 단위근 집합을 표현할 수 있음을 의미한다.

- 즉, 다음 두 식은 대칭적이고 효율적으로 계산될 수 있다.
  $$
  P(w_n^k) = P_{\text{even}}(w_n^{2k}) + w_n^k P_{\text{odd}}(w_n^{2k})
  $$
  $$
  P(w_n^{k + \frac{n}{2}}) = P_{\text{even}}(w_n^{2k}) - w_n^k P_{\text{odd}}(w_n^{2k})
  $$

- 이로 인해 단위근의 절반만 사용해도 전체 계산이 가능해지며, FFT는 재귀 단계마다 계산량을 절반으로 줄인다.

### IV-3) even/odd($x^2$)에서 보존되는 단위근의 재귀적 성질
#### 단위원 상에서 $w_n^k$들이 균등하게 회전하여 배치되는 모습
![roots of unity](../image/unity.png)
앞서 재귀 분할 시,

$P(x) = P_{\text{even}}(x^2) + x \cdot P_{\text{odd}}(x^2)$

여기서 $n$번째 단위근 $\omega_n$이 있을 때, $\omega_n^2$는 $\omega_{n/2}$에 해당한다.

왜냐하면:

$$
\omega_n = e^{\frac{2\pi i}{n}} \Rightarrow (\omega_n)^2 = e^{\frac{4\pi i}{n}} = e^{\frac{2\pi i}{n/2}} = \omega_{n/2}
$$

즉, 제곱 연산은 **각도를 2배**로 만들고, **분할 수는 절반**으로 줄어든다.  
이로 인해 $n$개의 단위근 중에서 **짝수 인덱스 단위근들**만으로 $n/2$개의 단위근이 재귀적으로 구성된다.


---

## V. 구현
이전 장에서 우리는 FFT가 짝수/홀수 항 분할(Divide)과 버터플라이 구조를 이용한 병합(Conquer)을 통해 DFT를 효율적으로 계산하는 재귀 알고리즘임을 확인했다. 이제 이를 실제 코드로 구현하는 과정을 단계별로 살펴본다.

#### **V-1. Cooley-Tukey 알고리즘**

본 보고서에서 설명한 재귀적 분할 정복 방식의 FFT 알고리즘은 1965년에 제임스 쿨리(James Cooley)와 존 튜키(John Tukey)에 의해 널리 알려졌으며, 이를 **쿨리-튜키(Cooley-Tukey) 알고리즘**이라 부른다. 이는 FFT를 구현하는 가장 대표적인 방법이다.

#### **V-2. 알고리즘 의사 코드 (Pseudo-code)**

쿨리-튜키 FFT 알고리즘의 재귀적 로직은 다음과 같은 의사 코드로 표현할 수 있다.

```
FUNCTION FFT(A):
    n = A의 길이
    IF n == 1, THEN RETURN A  // 기저 사례: 길이가 1이면 자기 자신을 반환

    // 1. 분할 (Divide)
    A_even = FFT(A의 짝수 인덱스 원소들)
    A_odd  = FFT(A의 홀수 인덱스 원소들)

    // 2. 병합 (Conquer)
    w_n = exp(2 * pi * i / n) // n차 단위근의 기본 요소
    w = 1                     // 회전 인자 (Twiddle factor) 초기화
    
    Y = 크기가 n인 배열
    FOR k FROM 0 TO n/2 - 1:
        t = w * A_odd[k]
        Y[k]       = A_even[k] + t  // 버터플라이 연산
        Y[k + n/2] = A_even[k] - t  // 버터플라이 연산
        w = w * w_n                 // 다음 회전 인자로 업데이트

    RETURN Y
```
이 의사 코드는 FFT의 핵심인 분할, 재귀 호출, 그리고 버터플라이 연산을 통한 병합 과정을 명확히 보여준다.

#### **V-3. 재귀 호출 흐름 시각화**

FFT의 재귀 호출 구조를 시각화하면 `O(n log n)` 복잡도를 직관적으로 이해할 수 있다. 예를 들어 8개의 계수를 가진 다항식에 대한 FFT는 다음과 같이 분해되고 병합된다.

![recursive](../image/fft1.png)  


위 그림에서 볼 수 있듯이, 크기 8의 문제는 크기 4의 문제 두 개로, 다시 크기 2의 문제 네 개로 분해된다. 이 재귀의 깊이는 $\log_2 n$이며, 각 깊이에서 총 $n$번의 연산이 수행되므로 전체 시간 복잡도는 $O(n \log n)$이 된다.

#### **V-4. 파이썬 구현: FFT와 IFFT**

이제 의사 코드를 바탕으로 실제 파이썬 코드를 작성한다. 복소수 연산을 위해 `numpy` 라이브러리를 사용한다.

```python
import numpy as np

def fft(a):
    """쿨리-튜키 FFT 알고리즘 구현"""
    n = len(a)
    if n == 1:
        return a

    # 짝수/홀수 부분으로 분할
    a_even = fft(a[0::2])
    a_odd = fft(a[1::2])

    # n차 단위근을 이용한 회전 인자(twiddle factor) 계산
    # w_n^k = exp(2 * pi * i * k / n)
    twiddle_factors = np.exp(2j * np.pi * np.arange(n // 2) / n)
    
    # 버터플라이 연산을 통한 병합
    y = np.zeros(n, dtype=np.complex128)
    # t = w * A_odd[k]
    t = twiddle_factors * a_odd
    y[:n // 2] = a_even + t
    y[n // 2:] = a_even - t
    
    return y

def ifft(y):
    """FFT를 이용한 IFFT 구현"""
    n = len(y)
    
    # 1. 켤레 복소수에 대해 FFT 수행
    # IFFT의 w^-k는 FFT의 w^k의 켤레 복소수와 같다.
    # fft(conj(y))는 IFFT 결과의 n배에 켤레를 취한 값과 같다.
    conjugate_y = np.conjugate(y)
    a_conjugate = fft(conjugate_y)
    
    # 2. 결과에 다시 켤레를 취하고 n으로 나눔
    a = np.conjugate(a_conjugate) / n
    return a
```
**IFFT 구현 원리**:
III-3장에서 보았듯이, IFFT는 DFT 행렬의 켤레 전치(conjugate transpose)를 사용하고 결과에 $1/n$을 곱하는 것과 같다. 이는 단위근 $\omega_n$ 대신 $\omega_n^{-1}$ (즉, $\overline{\omega_n}$)을 사용하고, 최종 결과를 $n$으로 나누는 것과 동일하다. 위 `ifft` 함수는 이러한 수학적 관계를 효율적으로 활용하여 FFT 함수를 재사용해 구현한 것이다.

#### **V-5. 실제 다항식 곱셈 예제 및 결과**

FFT와 IFFT를 이용하여 실제로 두 다항식을 곱하는 과정을 시연하고, 그 결과가 정확한지 검증한다.

-   **다항식 정의**:
    -   $A(x) = 1 + 2x + 3x^2$ (계수: `[1, 2, 3]`)
    -   $B(x) = 4 + 5x$ (계수: `[4, 5]`)
-   **정확한 곱셈 결과 (수계산)**:
    -   $C(x) = A(x)B(x) = (1 + 2x + 3x^2)(4 + 5x) = 4 + 13x + 22x^2 + 15x^3$
    -   기대 계수: `[4, 13, 22, 15]`

이제 FFT를 통해 이 결과를 도출해본다.

![recursive](../image/fft2.png) 

**결과 분석**:
위 코드를 실행하면, FFT를 통해 계산된 계수 `[4. 13. 22. 15.]`가 수계산으로 얻은 정확한 결과와 일치함을 확인할 수 있다. 또한, 시각화된 그래프에서 파란색 원(정확한 값)과 빨간색 X(FFT 결과)가 완벽하게 겹치는 것을 통해, FFT를 이용한 다항식 곱셈 알고리즘이 성공적으로 작동함을 증명할 수 있다. 이로써 $O(n \log n)$ 시간 복잡도를 갖는 고속 다항식 곱셈 구현이 완료되었다.